# First Classwork – Data Mining

Questo notebook affronta il problema legato al primo classwork di Data Mining. Per farlo, si utilizza principalmente una libreria esterna [TRACCIA](https://github.com/emanuelegaliano/TRACCIA). Il progetto è stato suddiviso in diverse fasi che possono essere trovate all'interno del file [roadmap.md](roadmap.md)

In [1]:
pip install -r requirements.txt # type: ignore

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/emanuelegaliano/TRACCIA.git to /tmp/pip-req-build-dmez_hmy
  Running command git clone --filter=blob:none --quiet https://github.com/emanuelegaliano/TRACCIA.git /tmp/pip-req-build-dmez_hmy
  Resolved https://github.com/emanuelegaliano/TRACCIA.git to commit 4dd9ba02a86286c7e28196f8a24696279416caf0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In questa cella importiamo:

- `Trail` dalla libreria **TRACCIA**, che ci permette di definire ed eseguire pipeline di elaborazione;
- gli **step di cleaning** definiti nel modulo `cleaning.py`, che verranno concatenati nel `CLEANING_TRAIL`;
- la classe `FirstClassworkFootprint`, che rappresenta lo stato condiviso tra tutti gli step della pipeline;
- la dataclass `Config`, che centralizza tutti i parametri di configurazione del progetto.

Questi import costituiscono la base per eseguire la **Fase B – Data Loading & Cleaning** dell’assignment.

**NB**: La fase A è intrisenca all'interno del codice sorgente, per questo viene saltata in questo notebook.

In [ ]:
from traccia import Trail
from src.pipelines.cleaning import (
    log_config,
    load_dataset,
    normalize_schema,
    parse_datetime,
    drop_invalid_rows,
    exclude_shoppers,
    add_time_bins,
    finalize_cleaning,
    CLEANING_TRAIL
)
from src.domain.footprint import FirstClassworkFootprint
from src.domain.config import Config

config = Config(
    data_path="AnonymizedFidelity.csv",
    output_dir="outputs",
    date_format="%Y-%m-%d",
    time_format=None,      # se non sai ancora se c'è :SS
    dayfirst=False         # coerente con ISO
)

fp = FirstClassworkFootprint(config=config)
CLEANING_TRAIL.trace_enabled = True # True for debug

fp = CLEANING_TRAIL.run(fp)

[TRACCIA] Trail 'Trail' starting
[TRACCIA] -> log_config
[TRACCIA] -> load_dataset
[TRACCIA] -> normalize_schema
[TRACCIA] -> parse_datetime
[TRACCIA] -> drop_invalid_rows
[TRACCIA] -> exclude_shoppers
[TRACCIA] -> add_time_bins
[TRACCIA] -> finalize_cleaning
[TRACCIA] Trail 'Trail' finished (handlers=['log_config', 'load_dataset', 'normalize_schema', 'parse_datetime', 'drop_invalid_rows', 'exclude_shoppers', 'add_time_bins', 'finalize_cleaning'])


## FASE B: Pipeline di cleaning

In questa cella:

1. Istanziamo l’oggetto `Config`, specificando:
   - il percorso del dataset (`AnonymizedFidelity.csv`),
   - la directory di output,
   - il formato delle date e delle ore (deterministico, per evitare warning e ambiguità).

2. Creiamo un oggetto `FirstClassworkFootprint`, che fungerà da contenitore condiviso
   per i dati grezzi, i dati puliti e i metadata di esecuzione.

3. Attiviamo la modalità `trace_enabled` sul `CLEANING_TRAIL` per visualizzare l’ordine
   di esecuzione degli step (utile in fase di debug).

4. Eseguiamo infine la pipeline di cleaning tramite il metodo `run`, ottenendo un
   dataset pulito e pronto per le successive fasi di analisi.

In [4]:
print(fp.raw_df[config.col_date].astype(str).head(10).tolist())
print(fp.raw_df[config.col_time].astype(str).head(10).tolist())

['2023-03-25', '2023-03-25', '2023-03-25', '2023-03-25', '2023-03-25', '2023-03-25', '2023-03-25', '2023-03-25', '2023-03-25', '2023-03-25']
['21:00', '21:00', '21:00', '21:00', '21:00', '21:00', '21:00', '21:00', '21:00', '21:00']


## Fase C - Analisi delle frequenze (Task 1)

In questa fase affrontiamo il **Task 1 dell’assignment**, che richiede di calcolare la frequenza
degli elementi per ciascun livello della gerarchia di merchandising e di visualizzare,
per ogni livello, i **Top-5** e **Bottom-5** elementi più frequenti.

### Obiettivi
- Calcolare le frequenze per i livelli:
  - `liv1`
  - `liv2`
  - `liv3`
  - `liv4`
- Generare, per ogni livello, due grafici a barre:
  - Top-5 elementi più frequenti
  - Bottom-5 elementi meno frequenti
- Salvare automaticamente i risultati su disco per l’inclusione nel report finale.

### Approccio
L’analisi è implementata come una **pipeline TRACCIA dedicata** (`TASK1_FREQUENCIES_TRAIL`),
che assume come input il dataset già pulito prodotto nella Fase B.

La pipeline esegue i seguenti step:
1. Calcolo delle tabelle di frequenza per ciascun livello di merchandising.
2. Estrazione dei Top-5 e Bottom-5 elementi.
3. Generazione e salvataggio dei grafici a barre in formato PNG.

Tutti i risultati intermedi e finali vengono memorizzati nel `Footprint`:
- le tabelle di frequenza sono salvate in `fp.freq_tables`,
- i percorsi dei grafici generati sono salvati in `fp.plots`.

I file di output sono scritti nella directory: [outputs/task1](outputs/task1)

In [5]:
from src.pipelines.frequencies_task1 import TASK1_FREQUENCIES_TRAIL

fp = TASK1_FREQUENCIES_TRAIL.run(fp)

fp.freq_tables["liv1"].head()
fp.plots[:5]

['outputs/task1/liv1_top5.png',
 'outputs/task1/liv1_bottom5.png',
 'outputs/task1/liv2_top5.png',
 'outputs/task1/liv2_bottom5.png',
 'outputs/task1/liv3_top5.png']

## Fase D – Analisi delle frequenze stratificate (Task 2)

In questa fase affrontiamo il **Task 2 dell’assignment**, che estende l’analisi descrittiva
del Task 1 introducendo una **stratificazione temporale** delle frequenze.

L’obiettivo è analizzare come la distribuzione degli elementi nei diversi livelli
di merchandising vari:
- durante differenti periodi dell’anno;
- nelle diverse fasce orarie della giornata.

### Stratificazioni considerate

Il dataset è stato precedentemente arricchito (Fase B) con due variabili temporali:

- **`month_range`**, che suddivide l’anno in tre periodi:
  - *Jan – mid May*
  - *mid May – Sep*
  - *Oct – Dec*

- **`time_slot`**, che suddivide la giornata in tre fasce orarie:
  - *08:30 – 12:30*
  - *12:30 – 16:30*
  - *16:30 – 20:30*

(Eventuali acquisti al di fuori di queste fasce vengono etichettati come `OUTSIDE`.)

### Approccio

L’analisi è implementata tramite una pipeline dedicata (`TASK2_FULL_TRAIL`), che:
1. Filtra il dataset per ciascun valore di stratificazione (`month_range` o `time_slot`);
2. Calcola le frequenze per ciascun livello di merchandising (`liv1`–`liv4`);
3. Estrae i **Top-5** e **Bottom-5** elementi per ogni combinazione livello–strato;
4. Genera e salva i grafici a barre corrispondenti.

Per mantenere il codice modulare e riutilizzabile, il calcolo delle frequenze e la
generazione dei plot sfruttano funzioni comuni definite nel modulo `utilities`.

### Output

I risultati vengono salvati automaticamente nella directory: [outputs/task2/](outputs/task2/)


con la seguente struttura:
- `outputs/task2/month_range/<periodo>/`
- `outputs/task2/time_slot/<fascia_oraria>/`

Le tabelle di frequenza e i percorsi dei grafici generati sono inoltre memorizzati
nel `Footprint`, permettendo un facile accesso ai risultati per il report finale.

In [6]:
from src.pipelines.frequencies_task2 import TASK2_FULL_TRAIL

TASK2_FULL_TRAIL.trace_enabled = True
fp = TASK2_FULL_TRAIL.run(fp)

fp.plots[-5:]

[TRACCIA] Trail 'Trail' starting
[TRACCIA] -> task2_month_range_frequencies_and_plots
[TRACCIA] -> task2_time_slot_frequencies_and_plots
[TRACCIA] Trail 'Trail' finished (handlers=['log_config', 'load_dataset', 'normalize_schema', 'parse_datetime', 'drop_invalid_rows', 'exclude_shoppers', 'add_time_bins', 'finalize_cleaning', 'task1_compute_frequencies', 'task1_plot_top_bottom', 'task2_month_range_frequencies_and_plots', 'task2_time_slot_frequencies_and_plots'])


['outputs/task2/time_slot/OUTSIDE/liv2_bottom5.png',
 'outputs/task2/time_slot/OUTSIDE/liv3_top5.png',
 'outputs/task2/time_slot/OUTSIDE/liv3_bottom5.png',
 'outputs/task2/time_slot/OUTSIDE/liv4_top5.png',
 'outputs/task2/time_slot/OUTSIDE/liv4_bottom5.png']

## Fase E – Association Rules Mining (Task 3 & Task 4)

In questa fase affrontiamo i **Task 3 e 4 dell’assignment**, che richiedono
l’estrazione di **regole di associazione** a partire dai dati di vendita,
utilizzando due algoritmi classici di data mining:

- **Apriori**
- **FP-Growth**

L’analisi viene condotta al **livello 4 della gerarchia di merchandising (liv4)**,
considerando ciascuno scontrino come una transazione e ciascun elemento di livello 4
come un item.

In [13]:
from src.pipelines.association_rules import TASK3_4_FULL_TRAIL

TASK3_4_FULL_TRAIL.trace_enabled = True
fp = TASK3_4_FULL_TRAIL.run(fp)

fp.transactions_lvl4.shape

[TRACCIA] Trail 'Trail' starting
[TRACCIA] -> build_transactions_lvl4
[TRACCIA] -> apriori_rules
[TRACCIA] -> fpgrowth_rules
[TRACCIA] -> save_rules
[TRACCIA] Trail 'Trail' finished (handlers=['log_config', 'load_dataset', 'normalize_schema', 'parse_datetime', 'drop_invalid_rows', 'exclude_shoppers', 'add_time_bins', 'finalize_cleaning', 'task1_compute_frequencies', 'task1_plot_top_bottom', 'task2_month_range_frequencies_and_plots', 'task2_time_slot_frequencies_and_plots', 'build_transactions_lvl4', 'apriori_rules', 'fpgrowth_rules', 'save_rules', 'build_transactions_lvl4', 'apriori_rules', 'fpgrowth_rules', 'save_rules'])


(149410, 141)

### Preprocessing per la scalabilità

A causa dell’elevata numerosità iniziale degli item di livello 4, è stato applicato
un filtro preliminare basato sul **supporto minimo**, mantenendo solo gli item
presenti in almeno il **2% delle transazioni** (`min_support = 0.02`).

Dopo questo filtraggio, la matrice transazioni–item risulta di dimensione: (149410 transazioni, 141 item)

### Parametri principali

- **Supporto minimo**: 0.02  
- **Confidenza minima**: 0.35  
- **Lift minimo**: 1.1  
- **Lunghezza massima delle regole**:
  - Apriori: 2
  - FP-Growth: fino a 3

I risultati delle due tecniche vengono confrontati in termini di numero e qualità
delle regole estratte.

In [14]:
fp.rules_apriori["support"].describe()

count    32.000000
mean      0.043502
std       0.016416
min       0.020039
25%       0.028315
50%       0.043180
75%       0.056777
max       0.073656
Name: support, dtype: float64

In [ ]:
fp.rules_apriori.sort_values("lift", ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
27,(3060404.0),(3060405.0),0.072458,0.081052,0.024081,0.332348,4.100423,1.0,0.018208,1.376388,0.815190,0.186059,0.273460,0.314729
1,(1150201.0),(1150202.0),0.137789,0.079178,0.043725,0.317336,4.007878,1.0,0.032815,1.348866,0.870426,0.252395,0.258636,0.434788
2,(1150202.0),(1150201.0),0.079178,0.137789,0.043725,0.552240,4.007878,1.0,0.032815,1.925611,0.815023,0.252395,0.480684,0.434788
20,(3060404.0),(3011512.0),0.072458,0.134844,0.031136,0.429706,3.186698,1.0,0.021365,1.517036,0.739800,0.176741,0.340820,0.330305
24,(3060404.0),(3060402.0),0.072458,0.216592,0.048417,0.668206,3.085093,1.0,0.032723,2.361129,0.728658,0.201207,0.576474,0.445873
28,(9010102.0),(9010101.0),0.100214,0.173917,0.050117,0.500100,2.875504,1.0,0.032688,1.652496,0.724878,0.223723,0.394855,0.394133
31,(9010102.0),(9010201.0),0.100214,0.130707,0.036363,0.362853,2.776071,1.0,0.023264,1.364352,0.711034,0.186900,0.267051,0.320527
25,(3060405.0),(3060402.0),0.081052,0.216592,0.048671,0.600495,2.772474,1.0,0.031116,1.960949,0.695699,0.195489,0.490043,0.412605
21,(3060405.0),(3011512.0),0.081052,0.134844,0.028485,0.351445,2.606314,1.0,0.017556,1.333975,0.670676,0.151995,0.250361,0.281346
23,(3060401.0),(3060402.0),0.064520,0.216592,0.036122,0.559855,2.584837,1.0,0.022147,1.779885,0.655416,0.147443,0.438166,0.363315


In [16]:
len(fp.rules_apriori), len(fp.rules_fpgrowth)

(32, 51)

## Analisi e interpretazione delle regole di associazione

### Dimensione dei risultati

L’algoritmo di **Apriori** ha prodotto un totale di: 32 regole di aassociazione mentre FP-Growth ha individuato un numero maggiore di pattern (non riportato qui per
brevità), confermando la sua maggiore efficienza nell’esplorazione di pattern più complessi.

Le 32 regole estratte presentano metriche statistiche stabili e ben distribuite,
come mostrato dalla distribuzione del supporto:
- minimo: 0.020
- massimo: 0.073
- media: 0.043

Questo indica che ogni regola è supportata da almeno il **2% degli scontrini**,
con alcune associazioni presenti in oltre il **7% delle transazioni**.

## Fase F – Riduzione dimensionale e clustering dei clienti (Task 5)

In questa fase affrontiamo il **Task 5 dell’assignment**, con l’obiettivo di individuare
gruppi omogenei di clienti sulla base dei loro comportamenti di acquisto.

L’analisi è condotta considerando esclusivamente le transazioni associate a una
**tessera fedeltà valida**, costruendo una matrice:

- righe → tessere cliente  
- colonne → prodotti acquistati  
- valori → frequenza di acquisto  

Data l’elevata dimensionalità della matrice risultante, viene applicata una
**riduzione dimensionale tramite PCA (Principal Component Analysis)**, seguita
da un algoritmo di **clustering non supervisionato**.

### Pipeline adottata

La pipeline implementata prevede i seguenti passaggi:

1. Selezione delle sole transazioni con tessera valida;
2. Costruzione della matrice tessera × prodotto;
3. Standardizzazione delle feature;
4. Applicazione della PCA, mantenendo l’80% della varianza totale;
5. Clustering dei clienti nello spazio PCA tramite **K-Means**.

Questo approccio consente di gestire un dataset di grandi dimensioni in modo
computazionalmente sostenibile, mantenendo al contempo l’informazione rilevante
per la segmentazione dei clienti.

In [17]:
from src.pipelines.pca_clustering import TASK5_PCA_CLUSTERING_TRAIL

TASK5_PCA_CLUSTERING_TRAIL.trace_enabled = True
fp = TASK5_PCA_CLUSTERING_TRAIL.run(fp)

[TRACCIA] Trail 'Trail' starting
[TRACCIA] -> filter_valid_cards
[TRACCIA] -> build_card_product_matrix
[TRACCIA] -> scale_matrix
[TRACCIA] -> apply_pca
[TRACCIA] -> cluster_cards
[TRACCIA] -> save_task5_outputs
[TRACCIA] Trail 'Trail' finished (handlers=['log_config', 'load_dataset', 'normalize_schema', 'parse_datetime', 'drop_invalid_rows', 'exclude_shoppers', 'add_time_bins', 'finalize_cleaning', 'task1_compute_frequencies', 'task1_plot_top_bottom', 'task2_month_range_frequencies_and_plots', 'task2_time_slot_frequencies_and_plots', 'build_transactions_lvl4', 'apriori_rules', 'fpgrowth_rules', 'save_rules', 'build_transactions_lvl4', 'apriori_rules', 'fpgrowth_rules', 'save_rules', 'filter_valid_cards', 'build_card_product_matrix', 'scale_matrix', 'apply_pca', 'cluster_cards', 'save_task5_outputs'])


In [26]:
print(fp.card_product_matrix.shape)
print(fp.metrics)

(9375, 19422)
{'task5_silhouette': 0.6868467882048566}


In [32]:
from sklearn.metrics import calinski_harabasz_score

print("Calinski-Harabasz Score:", calinski_harabasz_score(fp.pca_embeddings, fp.cluster_labels))

Calinski-Harabasz Score: 9.921387605182197


## Analisi e interpretazione dei risultati del clustering

### Dimensione del problema

Dopo il preprocessing, la matrice tessera × prodotto risulta di dimensione: (9375 tessere, 19422 prodotti).

Si tratta di un dataset ad **alta dimensionalità**, tipico di contesti retail reali,
caratterizzato da dati sparsi e fortemente sbilanciati.

### Riduzione dimensionale (PCA)

L’applicazione della PCA ha consentito di ridurre significativamente la dimensionalità
del problema, mantenendo l’80% della varianza totale con: 1739 componenti principali.


Questo rappresenta una riduzione di oltre il **90% delle feature originali**,
rendendo possibile l’applicazione efficace di algoritmi di clustering.

### Clustering

Il clustering è stato eseguito utilizzando **K-Means** nello spazio ridotto dalla PCA,
con un numero di cluster prefissato pari a 5.

La qualità della partizione è stata valutata tramite il **silhouette score**, che
assume il valore: 0.69.


Un valore di silhouette così elevato indica una **buona separazione tra i cluster**
e una struttura ben definita dei gruppi individuati, nonostante la complessità e
l’eterogeneità dei dati di partenza.

### Considerazioni sulla visualizzazione

La proiezione dei dati in due dimensioni (ad esempio sulle prime componenti principali
o tramite tecniche di embedding) non mostra una separazione netta dei cluster e pertanto non è stata fatta. 

Questo comportamento è atteso in presenza di dati ad alta dimensionalità, in cui la
struttura discriminante dei cluster risulta distribuita su un numero elevato di componenti
e non può essere efficacemente rappresentata in uno spazio bidimensionale senza perdita
significativa di informazione.

La qualità del clustering è stata quindi valutata principalmente attraverso **metriche
quantitative**, in particolare il **Calinski–Harabasz Score**, che assume nel nostro caso
il valore: Calinski–Harabasz Score = 9.92.

Un valore elevato di questa metrica indica un buon rapporto tra la dispersione
inter-cluster e quella intra-cluster, confermando la presenza di gruppi ben separati
nello spazio delle componenti principali, nonostante la separazione non sia facilmente
osservabile tramite proiezioni bidimensionali.